In [1]:
import numpy as np
import pandas as pd

from eva import EVStation

In [2]:
from tqdm import tqdm
import sys
import time
import timeit

In [3]:
sys.version

'3.9.12 | packaged by conda-forge | (main, Mar 24 2022, 23:24:38) \n[Clang 12.0.1 ]'

In [4]:
evs_config = {'N': 50000, 'Ns': 20,
              'Tagc': 4.0, 'socf': 0.2,
              'agc': 0, 'ict': 0,
              'memory_save': 1,
              'seed': 2022, 'r': 0.5}

mcs_config = {'ts': 18.0, 'h': 1.0,
              'no_tqdm': 0, 'th': 1}

nd_param = {'soci': {'mu': 0.3, 'var': 0.05, 'lb': 0.2, 'ub': 0.4},
            'socd': {'mu': 0.8, 'var': 0.03, 'lb': 0.7, 'ub': 0.9},
            'ts1': {'mu': -6.5, 'var': 3.4, 'lb': 0.0, 'ub': 5.5},
            'ts2': {'mu': 17.5, 'var': 3.4, 'lb': 5.5, 'ub': 24.0},
            'tf1': {'mu': 8.9, 'var': 3.4, 'lb': 0.0, 'ub': 20.9},
            'tf2': {'mu': 32.9, 'var': 3.4, 'lb': 20.9, 'ub': 24.0},
            'tt': {'mu': 0.5, 'var': 0.02, 'lb': 0, 'ub': 1}}

ud_param = {'Pc': {'lb': 5.0, 'ub': 7.0},
            'Pd': {'lb': 5.0, 'ub': 7.0},
            'nc': {'lb': 0.88, 'ub': 0.95},
            'nd': {'lb': 0.88, 'ub': 0.95},
            'Q': {'lb': 20.0, 'ub': 30.0}}


In [5]:
import logging
logger = logging.getLogger()
logging.getLogger("eva").setLevel(logging.INFO)

In [6]:
evs = EVStation(name='EVS1',
                mcs_config=mcs_config,
                config=evs_config,
                ud_param=ud_param,
                nd_param=nd_param)

Memory save is turned on, EVs out of time range [18.0, 19.0] are dropped.
EVS1: Initialized successfully with:
Capacity: 50000, r: 0.5
EVS1: clock time: 18.0, Online: 14477, Total: 17373


In [7]:
xdf = evs.MCS.data.d.copy()
xsf = evs.MCS.data.s.copy()

In [8]:
%%timeit -r 20 -n 20
xdf['u'] * xdf['c'] * xsf['Pc'] * xsf['nc'] * 1/3600

142 µs ± 21.7 µs per loop (mean ± std. dev. of 20 runs, 20 loops each)


In [9]:
xdf.columns

Index(['idx', 'u', 'u0', 'soc', 'c', 'lc', 'sx', 'na', 'ama', 'agc', 'mod'], dtype='object')

In [10]:
xsf.columns

Index(['idx', 'ts', 'tf', 'tt', 'soc0', 'na0', 'soci', 'socd', 'Pc', 'Pd',
       'nc', 'nd', 'Q', 'nam'],
      dtype='object')

In [11]:
xd = evs.MCS.data.d.values
xs = evs.MCS.data.s.values

In [12]:
%%timeit -r 20 -n 20
xd[:, 1] * xd[:, 4] * xs[:, 8] * xs[:, 10] * 1/3600

24.6 µs ± 12.6 µs per loop (mean ± std. dev. of 20 runs, 20 loops each)


In [ ]:
evs = EVStation(name='EVS1',
                mcs_config=mcs_config,
                config=evs_config,
                ud_param=ud_param,
                nd_param=nd_param)

evs.MCS.config.tf = 3600
evs.MCS.config.no_tqdm = 0
evs.MCS.run()
evs.MCS.data
evs.MCS.data.d.info(verbose=False, memory_usage=True)